# Black Hat USA Training (Early draft)

## Lab 2: Feature Extraction from Byte Level Data with Neural Networks and PyTorch

We will follow a "Top-Down" teaching methodology: We will start with higher level concepts familiar to our students in the cybersecurity domain, for instance, by introducing a specific library and demonstrating its use. Then, we delve deeper into the methods and parameters of these applications. Finally, we explore the underlying fundamentals, such as the specific PE format properties or mathematical concepts at the core of these ideas.

**NOTE: This is a raw draft that will be populated with more material (especially visual) and explanations, especially, facilitating AI/ML intuition and more gradual familiriaztion with concepts.**

Contents:
- Downloading AsyncRAT Sample
- Pre-Trained MalConv Model
- PyTorch Introduction
- PE File Path through Neural Network:
  - Embeddings
  - Convolutional Neural Network

### Downloading AsyncRAT Sample

We will use the same sample as in Lab 1:

In [353]:
# force reimport of lab_helpers
import sys
if 'lab_helpers' in sys.modules:
    del sys.modules['lab_helpers']

from lab_helpers import *

In [484]:
# NOTE: for some reason download from vx-underground is denied by the server 
# works from browser, but not if using requests.get, user-agent browser mimic does not help
vx_link = "https://samples.vx-underground.org/Samples/Families/AsyncRAT/5e3588e8ddebd61c2bd6dab4b87f601bd6a4857b33eb281cb5059c29cfe62b80.7z"
# using a private hosted copy
async_rat_path = "http://malware-training.us.to/5e3588e8ddebd61c2bd6dab4b87f601bd6a4857b33eb281cb5059c29cfe62b80.7z"
async_rat_bytez = get_encrypted_archive(async_rat_path, password="infected")
async_rat_bytez[0:20]

b'MZ\x90\x00\x03\x00\x00\x00\x04\x00\x00\x00\xff\xff\x00\x00\xb8\x00\x00\x00'

## Pre-Trained MalConv Model

MalConv is a binary classifier model that outputs a probability of the sample being malicious, proposed by group of researchers in this [paper](https://arxiv.org/abs/1710.09435). Under the hood it is a convolutional neural network (CNN) to extract features from the byte level of the malware sample. Schematic view of the model is as follows:

<img src="./img/malconv.png" width="600">

Let's download the pre-trained model and verify predictions on the AsyncRAT sample:

In [355]:
malconv_weights_link = "https://github.com/dtrizna/quo.vadis/raw/main/modules/sota/malconv/parameters/malconv.checkpoint"
malconv_weights = requests.get(malconv_weights_link).content
print(f"[+] Downloaded MalConv weights | Size: {len(malconv_weights) / 1024 / 1024:.2f} MB")

[+] Downloaded MalConv weights | Size: 24.79 MB


In [358]:
from lab_helpers import MalConvModel

torch.manual_seed(0)

malconv = MalConvModel()
malconv.load_state(malconv_weights)

score = malconv.get_score(async_rat_bytez)
print(f"[+] MalConv probability for Async RAT sample being malware: {score*100:.2f}%")

[+] MalConv probability for Async RAT sample being malware: 63.90%


In [359]:
malconv.model

MalConv(
  (embd): Embedding(257, 8, padding_idx=0)
  (conv_1): Conv1d(8, 256, kernel_size=(512,), stride=(512,))
  (conv_2): Conv1d(8, 256, kernel_size=(512,), stride=(512,))
  (pooling): AdaptiveMaxPool1d(output_size=1)
  (fc_1): Linear(in_features=256, out_features=256, bias=True)
  (fc_2): Linear(in_features=256, out_features=2, bias=True)
)

MalConv is neural network, and to understand how it works under the hood, we need to grasp basics of PyTorch.

## PyTorch Introduction

PyTorch is a Python library for implementing Deep Learning models. Deep Learning is a subfield of Machine Learning that uses **Neural Networks** to learn complex patterns in data. 

<img src="./img/ai_ml_dl.png" width="400">

During last years PyTorch became a de-facto standard for Deep Learning research, substituting the previous leader TensorFlow. PyTorch is a very flexible library that allows to implement complex models with a few lines of code.

### Tensors

Any deep learning framework operate with tensors, which are multi-dimensional arrays. In PyTorch, tensors are the main data structure. They are similar to NumPy arrays, but with additional features that make them suitable for deep learning:

<img src="./img/tensors.png" width="600">

In [287]:
import torch

# 1-D tensor (aka vector)
tensor_a = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
print(tensor_a.shape)
tensor_a 

torch.Size([9])


tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [288]:
# 2-D tensor (aka matrix)
tensor_b = tensor_a.reshape(3, 3)
print(tensor_b.shape)
tensor_b

torch.Size([3, 3])


tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

In [289]:
tensor_c = torch.vstack([tensor_a, tensor_a, tensor_a]).reshape(3, 3, 3)
print(tensor_c.shape)
tensor_c

torch.Size([3, 3, 3])


tensor([[[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]])

### Layers

#### Embeddings

Now, let's see how we can use PyTorch to extract features from the byte level of the malware sample. First, raw MZ file bytes are converted to an integer array:

In [291]:
import numpy as np

FIRST_N_BYTES = 5
torch.tensor( np.frombuffer(async_rat_bytez, dtype=np.uint8)[0:FIRST_N_BYTES].copy() )

tensor([ 77,  90, 144,   0,   3], dtype=torch.uint8)

We can confirm that the first bytes of the file are indeed the MZ header:

In [292]:
bytes([77, 90, 144, 0, 3])

b'MZ\x90\x00\x03'

In absolute majority of neural network architectures suited for textual analysis, first layer is an **Embedding** layer. Embeddings are basically a lookup table that maps each byte to a vector representation, learned during the training process. Important property of embedded vectors is that after training similar inputs are mapped to similar locations in the embedded vectorspace, as depicted in the following figure which displays **3-dimensional embeddings**:

<img src="./img/embedding_star_wars.gif" width="600">

[[Image Source]](https://medium.com/@marcusa314/visualizing-words-377624cb20c7)

MalConv uses **embedding size of 8**, but it is common to use higher dimensionality in modern models, such as 64 or 128.

Let's define an 8-dimensional embedding layer and pass the first 5 bytes of the AsyncRAT sample through it:

In [362]:
# embedding that encodes each byte to 3 dimensions, 256 possible values
nr_of_bytes = 256
embedding_size = 8
torch.manual_seed(0)
example_embed = torch.nn.Embedding(nr_of_bytes, embedding_size)

# get the embedding for the first 5 bytes of the Async RAT sample
async_first_5_bytez = torch.tensor([77, 90, 144, 0, 3])
print(f"[!] Shape of data before embedding: {async_first_5_bytez.shape}")

[!] Shape of data before embedding: torch.Size([5])


In the embedded array each byte is expanded to a 8-dimensional vector:

In [363]:
print(f"[!] Shape of data after embedding: {example_embed(async_first_5_bytez).shape}")

[!] Shape of data after embedding: torch.Size([5, 8])


In [365]:
print(f"[!] Data after embedding:")

example_embed(async_first_5_bytez)

[!] Data after embedding:


tensor([[-0.7650, -0.4750, -0.4953, -0.1984,  2.2149, -0.1367, -1.0182,  0.1784],
        [ 0.7049,  0.0305, -0.8542,  0.5388, -0.5265, -1.3320,  1.5451,  0.4086],
        [ 0.4047, -0.6549,  0.0521,  0.3401, -0.2124,  1.5629, -0.9072, -1.5662],
        [-1.1258, -1.1524, -0.2506, -0.4339,  0.8487,  0.6920, -0.3160, -2.1152],
        [ 0.7502, -0.5855, -0.1734,  0.1835,  1.3894,  1.5863,  0.9463, -0.8437]],
       grad_fn=<EmbeddingBackward0>)

In [367]:
first_byte = async_first_5_bytez[0]
with torch.no_grad():
    first_byte_embed = example_embed(async_first_5_bytez)[0]

print(f"[!] First byte: {first_byte}")
print(f"[!] First byte's embedding: {first_byte_embed}")

[!] First byte: 77
[!] First byte's embedding: tensor([-0.7650, -0.4750, -0.4953, -0.1984,  2.2149, -0.1367, -1.0182,  0.1784])


The idea should be clear by now. Now, we need more representative information size -- take **first 200000 bytes** of the AsyncRAT sample and pass it through the embedding layer:

In [371]:
asyncrat_tensor = torch.tensor(np.frombuffer(async_rat_bytez, dtype=np.uint8)[np.newaxis,:].copy())[:, :200000]
asyncrat_tensor.shape

torch.Size([1, 200000])

In [372]:
asyncrat_embedded = example_embed(asyncrat_tensor.long())
asyncrat_embedded.shape

torch.Size([1, 200000, 8])

#### Convolutional Layer

In MalConv, the output of the embedding layer is passed to a 1D convolutional layer, that take a raw byte sequence and extracts features from the byte sequence by applying a filter to a window of bytes at a time.

1D convolutional example below depicts input with **embedding size** of **3**, and convolution having **kernel size** is **3** with **stride** of **1**:

<img src="./img/conv_1D_time.gif" width="600">

We need to transpose the array to match the input shape of the convolutional layer, which expects the input dimensions to be:

`(batch_size, embedding_size, sequence_length)`

In [390]:
# switch the 1st and 2nd dimensions
asyncrat_embedded_prep = torch.transpose(asyncrat_embedded, 2, 1)

print(f"[!] Shape of data before 1D convolutional layer: {asyncrat_embedded_prep.shape}")

[!] Shape of data before 1D convolutional layer: torch.Size([1, 8, 200000])


MalConv uses 1D convolutional layer with a **kernel size of 512**, applies 256 filters (number of independent convolutional extractors), and uses **stride** of **512**:

In [403]:
torch.manual_seed(0)
conv_layer = torch.nn.Conv1d(in_channels=8, out_channels=256, kernel_size=512, stride=512)

Because of the stride, sequence length from original 200000 bytes are reduced to 390, with 256 independent convolutions applied to each window of 512 bytes:

In [404]:
print(f"[!] Shape of data after 1D convolutional layer: {conv_layer(asyncrat_embedded_prep).shape}")

[!] Shape of data after 1D convolutional layer: torch.Size([1, 256, 390])


In [405]:
conv_layer(asyncrat_embedded_prep)

tensor([[[-0.3150,  0.1721,  0.0220,  ...,  0.9214,  0.5167, -0.5706],
         [ 0.2943, -0.6932,  0.0379,  ...,  0.1647, -0.0668, -0.0521],
         [ 0.0768, -0.9937,  0.2447,  ...,  0.3427, -0.8050, -0.1158],
         ...,
         [-0.0962, -0.2683,  1.5979,  ..., -0.9385,  0.0210, -1.0429],
         [ 0.5865, -0.1245,  0.1460,  ..., -0.9941,  0.0721, -0.5876],
         [-0.0677, -0.4522, -1.1285,  ...,  0.5529,  0.1676, -0.9465]]],
       grad_fn=<ConvolutionBackward0>)

Each element in this output tensor is single number representation of 512 bytes of the original sample, extracted by the convolutional layer.

This tensor is then passed through a max pooling layer, which takes the maximum value from each filter output, reducing the tensor to a single dimension.

### Linear Layers

Finally, the output is passed through a fully connected (aka **Linear**) layers, which is a standard neural network living in everyone heads:

<img src="./img/linear.png" width="300">

Linear layers can be considered as knowledge base of the model. These layers learn convoluted feature mapping to an actual label, and are used to make the final prediction of the sample being malicious or benign.

MalConv stacks two linear layers with **ReLU** activation function in between, and a final linear layer with **Sigmoid** activation function to output the probability of the sample being malicious:

In [435]:
torch.manual_seed(0)
print(f"[!] Shape of data after 1D conv. layer:\t     {conv_layer(asyncrat_embedded_prep).shape}")
pooling = nn.AdaptiveMaxPool1d(1)
pooled = pooling(conv_layer(asyncrat_embedded_prep))
print(f"[!] Shape of data after max pooling:\t     {pooled.shape}")

linear_1 = nn.Linear(256, 256)
linear_1_out = torch.relu(linear_1(pooled.view(-1, 256)))
print(f"[!] Shape of data after first linear layer:  {linear_1_out.shape}")

linear_2 = nn.Linear(256, 1)
logit = linear_2(linear_1_out)
print(f"[!] Shape of data after second linear layer: {logit.shape}")

probability = torch.sigmoid(logit)
print(f"\n[!] Final probability of Async RAT sample being malware:\n\n{probability[0].item()*100:.2f}%")

[!] Shape of data after 1D conv. layer:	     torch.Size([1, 256, 390])
[!] Shape of data after max pooling:	     torch.Size([1, 256, 1])
[!] Shape of data after first linear layer:  torch.Size([1, 256])
[!] Shape of data after second linear layer: torch.Size([1, 1])

[!] Final probability of Async RAT sample being malware:

62.57%


## Defining a `torch` Neural Model

Let's put all these layers together to form an actual Neural Network model that can learn byte level features from the PE samples and identify malicious patterns. We will use the same model as in the original MalConv paper which uses few extra additions to previously described components.

In [436]:
class MalConv(nn.Module):
    # trained to minimize cross-entropy loss: criterion = nn.CrossEntropyLoss()
    def __init__(
            self,
            embd_size=8, # dimensionality of the byte embeddings
            total_nr_of_bytes=256, # number of possible byte values
            channels=256, # number of independent channels in the convolutional layer
            window_size=512, # size of the convolutional window
            stride=512, # stride (jump length) of the convolutional window
            out_size=2 # size of the output layer, corresponds to the number of classes we want to detect
    ):
        super(MalConv, self).__init__()
        bytes_with_padding = total_nr_of_bytes + 1
        self.embd = nn.Embedding(bytes_with_padding, embd_size, padding_idx=0)
        
        self.window_size = window_size
    
        self.conv_1 = nn.Conv1d(embd_size, channels, window_size, stride=stride, bias=True)
        self.conv_2 = nn.Conv1d(embd_size, channels, window_size, stride=stride, bias=True)
        
        self.pooling = nn.AdaptiveMaxPool1d(1)
        
        self.fc_1 = nn.Linear(channels, channels)
        self.fc_2 = nn.Linear(channels, out_size)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embd(x.long())
        x = torch.transpose(x, 2, 1)
        
        cnn_value = self.conv_1(x)
        gating_weight = torch.sigmoid(self.conv_2(x))
        
        x = cnn_value * gating_weight
        
        x = self.pooling(x)
        
        x = x.view(x.size(0), -1) # flatten
        
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        
        return x

In [437]:
torch.manual_seed(0)

with torch.no_grad():
    malconv = MalConv()
    logits = malconv(asyncrat_tensor)

logits.shape

torch.Size([1, 2])

In [438]:
print(f"[+] MalConv probability for Async RAT sample being malware: {torch.sigmoid(logits)[0, 1].item()*100:.2f}%")

[+] MalConv probability for Async RAT sample being malware: 47.52%


We haven't loaded the pre-trained model yet -- this is output from the randomly initialized model. 

Let's load the pre-trained model and verify predictions on the AsyncRAT sample:

In [479]:
import io

malconv_weights_dict = torch.load(io.BytesIO(malconv_weights))
malconv.load_state_dict(malconv_weights_dict['model_state_dict'])

<All keys matched successfully>

In [480]:
async_rat_bytez_200k = async_rat_bytez[:2000000]
asyncrat_tensor = torch.tensor(np.frombuffer(async_rat_bytez_200k, dtype=np.uint8)[np.newaxis,:].copy())

with torch.no_grad():
    outputs = torch.softmax(malconv(asyncrat_tensor), dim=-1)

outputs.detach().numpy()[0,1]

0.6389805

We can try to change random seed and verify that for the non-trained model probability changes, but stays highly uncertain, somewhere close to 50% all the time:

In [478]:
for seed in range(5):
    torch.manual_seed(seed)
    malconv = MalConv()
    logits = malconv(asyncrat_tensor)
    print(f"[!] Seed: {seed}")
    print(f"\tProbability AsyncRAT: {torch.softmax(logits, dim=-1)[0, 1].item()*100:>8.2f}%")

    if os.path.exists(r"C:\windows\system32\notepad.exe"):
        with open (r"C:\windows\system32\notepad.exe", "rb") as f:
            legit_bytez = f.read()
        legit_tensor = torch.tensor(np.frombuffer(legit_bytez, dtype=np.uint8)[np.newaxis,:].copy())[:, :200000]
        logits = malconv(legit_tensor)
        print(f"\tProbability notepad.exe: {torch.softmax(logits, dim=-1)[0, 1].item()*100:>5.2f}%")

[!] Seed: 0
	Probability AsyncRAT:    48.19%
	Probability notepad.exe: 46.97%
[!] Seed: 1
	Probability AsyncRAT:    52.60%
	Probability notepad.exe: 53.48%
[!] Seed: 2
	Probability AsyncRAT:    52.01%
	Probability notepad.exe: 52.85%
[!] Seed: 3
	Probability AsyncRAT:    54.57%
	Probability notepad.exe: 49.66%
[!] Seed: 4
	Probability AsyncRAT:    46.05%
	Probability notepad.exe: 48.22%


Pre-trained model, on the other hand, has the same score and differentiates between malicious and benign samples:

In [482]:
for seed in range(5):
    torch.manual_seed(seed)
    malconv = MalConv()

    # load the pre-trained weights
    malconv_weights_dict = torch.load(io.BytesIO(malconv_weights))
    malconv.load_state_dict(malconv_weights_dict['model_state_dict'])
    
    logits = malconv(asyncrat_tensor)
    print(f"[!] Seed: {seed}")
    print(f"\tProbability malicious AsyncRAT: {torch.softmax(logits, dim=-1)[0, 1].item()*100:>8.2f}%")

    if os.path.exists(r"C:\windows\system32\notepad.exe"):
        with open (r"C:\windows\system32\notepad.exe", "rb") as f:
            legit_bytez = f.read()
        legit_tensor = torch.tensor(np.frombuffer(legit_bytez, dtype=np.uint8)[np.newaxis,:].copy())[:, :200000]
        logits = malconv(legit_tensor)
        print(f"\tProbability malicious notepad.exe: {torch.softmax(logits, dim=-1)[0, 1].item()*100:>5.2f}%")

[!] Seed: 0
	Probability malicious AsyncRAT:    63.90%
	Probability malicious notepad.exe:  5.63%
[!] Seed: 1
	Probability malicious AsyncRAT:    63.90%
	Probability malicious notepad.exe:  5.63%
[!] Seed: 2
	Probability malicious AsyncRAT:    63.90%
	Probability malicious notepad.exe:  5.63%
[!] Seed: 3
	Probability malicious AsyncRAT:    63.90%
	Probability malicious notepad.exe:  5.63%
[!] Seed: 4
	Probability malicious AsyncRAT:    63.90%
	Probability malicious notepad.exe:  5.63%


Note, MalConv is still research prototype and these pre-trained weights are not usable in production environment:

In [483]:
if os.path.exists(r"C:\windows\system32\calc.exe"):
    with open (r"C:\windows\system32\calc.exe", "rb") as f:
        legit_bytez = f.read()

    malconv = MalConv()

    # load the pre-trained weights
    malconv_weights_dict = torch.load(io.BytesIO(malconv_weights))
    malconv.load_state_dict(malconv_weights_dict['model_state_dict'])

    legit_tensor = torch.tensor(np.frombuffer(legit_bytez, dtype=np.uint8)[np.newaxis,:].copy())[:, :200000]
    logits = malconv(legit_tensor)
    print(f"[+] Probability malicious calc.exe: {torch.softmax(logits, dim=-1)[0, 1].item()*100:.2f}%")

[+] Probability malicious calc.exe: 98.65%


# Explainability

We will now explore the model's predictions and try to understand why it classified the AsyncRAT sample as malicious.

In [443]:
#TODO